In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [4]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images.shape

(60000, 28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 07m 42s]
val_accuracy: 0.8978333473205566

Best val_accuracy So Far: 0.9039999842643738
Total elapsed time: 00h 42m 11s
INFO:tensorflow:Oracle triggered exit


In [19]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        38448     
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 64)                1228864   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,268,794
Trainable params: 1,268,794
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 124s 73ms/step - loss: 0.1851 - accuracy: 0.9300 - val_loss: 0.2713 - val_accuracy: 0.9067
Epoch 5/10
1688/1688 [==============================] - 145s 86ms/step - loss: 0.1474 - accuracy: 0.9442 - val_loss: 0.2805 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 104s 62ms/step - loss: 0.1169 - accuracy: 0.9556 - val_loss: 0.3015 - val_accuracy: 0.9110
Epoch 7/10
1688/1688 [==============================] - 103s 61ms/step - loss: 0.0935 - accuracy: 0.9645 - val_loss: 0.3542 - val_accuracy: 0.9000
Epoch 8/10
1688/1688 [==============================] - 103s 61ms/step - loss: 0.0759 - accuracy: 0.9714 - val_loss: 0.3631 - val_accuracy: 0.9090
Epoch 9/10
1688/1688 [==============================] - 104s 61ms/step - loss: 0.0606 - accuracy: 0.9776 - val_loss: 0.3786 - val_accuracy: 0.9017
Epoch 10/10
1688/1688 [==============================] - 104s 61ms/step - loss: 0.0496 - accuracy: 0.9808 - val_loss: 